# Top 5 Artists

Assume there are three Spotify tables: `artists`, `songs`, & `global_song_rank`, which contain information about the artists, songs, & music charts.

Write a query to find the top 5 artists whose songs appear most frequently in the Top 10 of the `global_song_rank` table. Display the top 5 artist names in ascending order, along with their song appearance ranking.

If two or more artists have the same number of song appearances, they should be assigned the same ranking, & the rank numbers should be continuous (i.e., 1, 2, 2, 3, 4, 5).

# Answer

Jesus that's a lot of tables. Here is the `artists` table:

```
CREATE TABLE artists (
	artist_id smallint,
	artist_name text,
	label_owner text
);

COPY artists
FROM '/Users/jiehengyu/Desktop/DataLemur SQL Interview Questions/Spotify - Top 5 Artists/artists.csv'
WITH (FORMAT CSV, HEADER);

SELECT * FROM artists;
```

<img src = "artists Table.png" width = "600" style = "margin:auto"/>

One down, next is the `songs` table:

```
CREATE TABLE songs (
	song_id integer,
	artist_id smallint,
	name text
);

COPY songs
FROM '/Users/jiehengyu/Desktop/DataLemur SQL Interview Questions/Spotify - Top 5 Artists/songs.csv'
WITH (FORMAT CSV, HEADER, QUOTE '"');

SELECT * FROM songs;
```

<img src = "songs Table.png" width = "600" style = "margin:auto"/>

Last but not least is the `global_song_rank` table:

```
CREATE TABLE global_song_rank (
	day smallint,
	song_id integer,
	rank smallint
);

COPY global_song_rank
FROM '/Users/jiehengyu/Desktop/DataLemur SQL Interview Questions/Spotify - Top 5 Artists/global_song_rank.csv'
WITH (FORMAT CSV, HEADER);

SELECT * FROM global_song_rank;
```

<img src = "global_song_rank Table.png" width = "600" style = "margin:auto"/>

This query is asking for a lot. I'll break it down to the artists who have had at least 1 song that was ranked top 10 globally.

```
WITH top10_songs_global
AS (
	SELECT global_song_rank.day,
		   artists.artist_name,
		   songs.name,
		   global_song_rank.rank
	FROM global_song_rank
	LEFT JOIN songs
		ON global_song_rank.song_id = songs.song_id
	LEFT JOIN artists
		ON songs.artist_id = artists.artist_id
	WHERE global_song_rank.rank <= 10
)
SELECT artist_name,
	   count(*)
FROM top10_songs_global
GROUP BY artist_name
ORDER BY count(*) DESC;
```

<img src = "Top 10 Global Artists.png" width = "600" style = "margin:auto"/>

Now we only need the top 5 of the top 10, so we'll rank the artists based on the number of songs they've had ranked top 10 globally.

```
SELECT *
FROM (
	WITH top10_songs_global
	AS (
		SELECT global_song_rank.day,
			   artists.artist_name,
			   songs.name,
			   global_song_rank.rank
		FROM global_song_rank
		LEFT JOIN songs
			ON global_song_rank.song_id = songs.song_id
		LEFT JOIN artists
			ON songs.artist_id = artists.artist_id
		WHERE global_song_rank.rank <= 10
	)
	SELECT artist_name,
		   count(*),
		   dense_rank() OVER (ORDER BY count(*) DESC)
	FROM top10_songs_global
	GROUP BY artist_name
	ORDER BY count(*) DESC
)
WHERE dense_rank <= 5;
```

<img src = "Top 5 Globally Ranked Artists.png" width = "600" style = "margin:auto"/>

Taylor Swift is the top artist, with 8 songs cracking the top 10 globally. Tied in second place is Bad Bunny & Drake, each with 7 songs within the top 10 global rankings. Third is also tied, between Ed Sheeran & Adele, each with 5 songs in the top 10 global rankings. 4th is Lady Gaga & 5th is Katy Perry, with 3 & 2 songs ever ranked top 10 globally.